In [5]:
!pip install elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 6.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 9.6 MB/s eta 0:00:00


In [6]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

es = Elasticsearch("http://elasticsearch:9200",
                    basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()

ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': 'UedtrHDrQR-yjuAD2s_9RA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
# TODO import documents into elastic

import requests
import json

index_name = "texts"

if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

output_file = "inverted_index.json" 

with open(output_file, 'r') as file:
    inverted_index = json.load(file)

for word, doc_ids in inverted_index.items():
    document = {
        "word": word,
        "document_ids": doc_ids
    }
    response = requests.post(f"{es_endpoint}/{index_name}/_doc", headers={"Content-Type": "application/json"}, data=json.dumps(document))
    if response.status_code == 201:
        print(f"Indexed document for word '{word}' successfully.")
    else:
        print(f"Failed to index document for word '{word}': {response.status_code} - {response.text}")


Indexed document for word 'wonderful' successfully.
Indexed document for word 'serenity' successfully.
Indexed document for word 'taken' successfully.
Indexed document for word 'possession' successfully.
Indexed document for word 'entire' successfully.
Indexed document for word 'soul' successfully.
Indexed document for word 'sweet' successfully.
Indexed document for word 'mornings' successfully.
Indexed document for word 'spring' successfully.
Indexed document for word 'enjoy' successfully.
Indexed document for word 'whole' successfully.
Indexed document for word 'heart' successfully.
Indexed document for word 'alone' successfully.
Indexed document for word 'feel' successfully.
Indexed document for word 'charm' successfully.
Indexed document for word 'existence' successfully.
Indexed document for word 'spot' successfully.
Indexed document for word 'created' successfully.
Indexed document for word 'bliss' successfully.
Indexed document for word 'souls' successfully.
Indexed document for

In [18]:
# TODO Query "money"

query = {
    "query": {
        "match": {
            "word": "money"  
        }
    }
}

response = requests.get(f"{es_endpoint}/{index_name}/_search", headers={"Content-Type": "application/json"}, data=json.dumps(query))

if response.status_code == 200:
    results = response.json()
    print("Query results:")
    for hit in results['hits']['hits']:
        print(hit['_source'])
else:
    print(f"Failed to perform query: {response.status_code} - {response.text}")

Query results:
{'word': 'silk', 'document_ids': ['Text3.txt']}
{'word': 'pyjamas', 'document_ids': ['Text3.txt']}


In [10]:
# TODO Query "money *"

query = {
    "query": {
        "wildcard": {
            "word": {
                "value": "money*"
            }
        }
    }
}

response = requests.get(f"{es_endpoint}/{index_name}/_search", headers={"Content-Type": "application/json"}, data=json.dumps(query))

if response.status_code == 200:
    results = response.json()
    print("Query results:")
    for hit in results['hits']['hits']:
        print(hit['_source'])
else:
    print(f"Failed to perform query: {response.status_code} - {response.text}")


Query results:
{'word': 'money', 'document_ids': ['Text2.txt']}


In [13]:
# TODO Query "money for charity"

query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"content": "money"}},
                {"match": {"content": "charity"}}
            ]
        }
    }
}

response = requests.get(f"{es_endpoint}/{index_name}/_search", headers={"Content-Type": "application/json"}, data=json.dumps(query))

if response.status_code == 200:
    results = response.json()
    print("Query results:")
    for hit in results['hits']['hits']:
        print(hit['_source'])
else:
    print(f"Failed to perform query: {response.status_code} - {response.text}")


Query results:


In [15]:
# TODO Query "money for charity" in an alternative way

alternative_query = {
    "query": {
        "bool": {
            "should": [
                {"match": {"content": "money"}},
                {"match": {"content": "charity"}}
            ]
        }
    }
}


response = requests.get(f"{es_endpoint}/{index_name}/_search", headers={"Content-Type": "application/json"}, data=json.dumps(alternative_query))

if response.status_code == 200:
    results = response.json()
    print("Query results for alternative match:")
    for hit in results['hits']['hits']:
        print(hit['_source'])
else:
    print(f"Failed to perform query: {response.status_code} - {response.text}")


Query results for alternative match:


In [ ]:
# TODO Query "silk pyjamas" e)

query = {
    "query": {
        "match": {
            "word": "silk pyjamas"  
        }
    }
}

response = requests.get(f"{es_endpoint}/{index_name}/_search", headers={"Content-Type": "application/json"}, data=json.dumps(query))

if response.status_code == 200:
    results = response.json()
    print("Query results:")
    for hit in results['hits']['hits']:
        print(hit['_source'])
else:
    print(f"Failed to perform query: {response.status_code} - {response.text}")